# Laboratorio 3 - Data Science Clasificación de rótulos de tráfico utilizando CNN Le-Net
Javier Ramírez - 21600  
Mario Cristales - 21631

In [58]:
import tensorflow as tf
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, f1_score
from tensorflow.keras import layers, models

## 1. Preparación de los datos

In [59]:
# Cargar el archivo de entrenamiento
with open('entrenamiento.p', 'rb') as file:
    data = pickle.load(file)

# Inspeccionar las llaves del diccionario
print(data.keys())

# Inspeccionar los tipos de datos de las llaves
for key in data.keys():
    print(f"Key: {key}, Type: {type(data[key])}, Length: {len(data[key])}")


dict_keys(['coords', 'labels', 'features', 'sizes'])
Key: coords, Type: <class 'numpy.ndarray'>, Length: 34799
Key: labels, Type: <class 'numpy.ndarray'>, Length: 34799
Key: features, Type: <class 'numpy.ndarray'>, Length: 34799
Key: sizes, Type: <class 'numpy.ndarray'>, Length: 34799


In [60]:
# Cargar los datos de entrenamiento
def load_data(pickle_file):
    with open(pickle_file, 'rb') as file:
        data = pickle.load(file)
    return data['features'], data['labels']

In [61]:
# Cargar los datos de entrenamiento, validación y prueba
X_train, y_train = load_data('entrenamiento.p')
X_val, y_val = load_data('validacion.p')
X_test, y_test = load_data('prueba.p')

In [62]:
# Normalizar las imágenes
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Convertir las etiquetas a arreglos de numpy
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# Convertir las etiquetas a one-hot encoding
X_train = tf.image.rgb_to_grayscale(X_train)
X_val = tf.image.rgb_to_grayscale(X_val)
X_test = tf.image.rgb_to_grayscale(X_test)

In [63]:
# visualizar las dimensiones de los datos
print(f'Train data shape: {X_train.shape}, Labels shape: {y_train.shape}')
print(f'Validation data shape: {X_val.shape}, Labels shape: {y_val.shape}')
print(f'Test data shape: {X_test.shape}, Labels shape: {y_test.shape}')

Train data shape: (34799, 32, 32, 1), Labels shape: (34799,)
Validation data shape: (4410, 32, 32, 1), Labels shape: (4410,)
Test data shape: (12630, 32, 32, 1), Labels shape: (12630,)


## 2. Implementación de la arquitectura Le-Net:

### 1. Presentación de la Arquitectura LeNet

**LeNet** es una de las arquitecturas pioneras en el ámbito de las Redes Neuronales Convolucionales (CNNs), desarrollada por Yann LeCun y su equipo en 1998. Fue diseñada específicamente para la tarea de reconocimiento de dígitos escritos a mano, como los presentes en el conjunto de datos MNIST. LeNet demostró ser efectiva en la extracción automática de características relevantes a partir de imágenes, reduciendo la necesidad de preprocesamiento manual.

- **Conv1**: La primera capa convolucional aplica 6 filtros de 5x5 a las imágenes de entrada de tamaño 32x32 píxeles. Como resultado, se generan 6 mapas de características de 28x28, debido a la reducción en tamaño por los bordes.
- **Pool1**: A continuación, se aplica una capa de Max-pooling con ventanas de 2x2 y un stride de 2, lo que reduce las dimensiones de los mapas de características a 14x14. Esta reducción de dimensionalidad ayuda a disminuir la carga computacional y a resumir las características más importantes.
- **Conv2**: La segunda capa convolucional aplica 16 filtros de 5x5 a los mapas de características resultantes, generando 16 mapas de características de 10x10. Esta capa profundiza la capacidad de la red para extraer características más complejas y específicas.
- **Pool2**: Similar a la primera capa de pooling, se aplica Max-pooling con ventanas de 2x2, reduciendo los mapas de características a 5x5. Este proceso continúa condensando la información relevante.
- **FC1**: La salida de la segunda capa de pooling se aplana y se pasa a una capa completamente conectada con 120 neuronas. En esta capa, la red combina las características extraídas para formar representaciones más abstractas.
- **FC2**: La siguiente capa completamente conectada tiene 84 neuronas, donde se realiza una mayor combinación y refinamiento de las características.
- **Capa de Salida**: Finalmente, la red termina con una capa de salida que contiene 10 neuronas, correspondientes a las 10 clases posibles de dígitos (0-9). Esta capa utiliza softmax para generar probabilidades de clasificación.

### 2. Diseño de la Red LeNet
El diseño de LeNet sigue un patrón estructurado de capas convolucionales y de pooling, seguidas por capas completamente conectadas. Esta arquitectura permitió que LeNet fuera uno de los primeros modelos en lograr una alta precisión en tareas de reconocimiento visual.
![Diagrama de Arquitectura](diagrama.png)

### 3. Proceso de Convolución y Pooling

- **Convolución**: 
  - **Extracción de Características Locales**: Los filtros convolucionales, también llamados kernels, actúan como detectores de características. Al deslizarse por la imagen, estos filtros capturan patrones locales como bordes, texturas, y formas simples.
  - **Función de Activación (ReLU)**: Tras la convolución, se aplica la función de activación ReLU (Rectified Linear Unit), que introduce no linealidad al modelo. Esta no linealidad es crucial para permitir que la red aprenda representaciones complejas.
  
- **Pooling**:
  - **Max-Pooling**: Este proceso selecciona el valor máximo dentro de una ventana de tamaño fijo, como 2x2. Max-pooling reduce la dimensionalidad de los mapas de características, conservando solo la información más relevante. Esto no solo ayuda a reducir la complejidad computacional, sino que también agrega una forma de invarianza a la traslación, haciendo que la red sea más robusta ante pequeñas variaciones en la posición de las características detectadas.



## 3. Construcción del modelo:

In [64]:
# Crear el modelo
model = models.Sequential()

# Capa de convolución 1
model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Capa de convolución 2
model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
model.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Aplanar las salidas
model.add(layers.Flatten())

# Capa completamente conectada 1
model.add(layers.Dense(units=120, activation='relu'))

# Capa completamente conectada 2
model.add(layers.Dense(units=84, activation='relu'))

# Capa de salida
model.add(layers.Dense(units=43, activation='softmax'))

# Resumen del modelo
model.summary()


c:\Users\javie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_6             │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_7             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 120)            │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 43)             │         3,655 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,511 (252.00 KB)

 Trainable params: 64,511 (252.00 KB)

 Non-trainable params: 0 (0.00 B)

La **función de pérdida** mide qué tan bien el modelo está realizando la tarea para la que fue entrenado, comparando las predicciones con las etiquetas verdaderas. Su objetivo es cuantificar el error del modelo, permitiendo ajustes durante el entrenamiento para mejorar su precisión.

El **optimizador** es un algoritmo que ajusta los parámetros del modelo para minimizar la función de pérdida. Utiliza el gradiente descendente y otras técnicas para encontrar los valores óptimos de los pesos y sesgos del modelo, mejorando así su rendimiento en la tarea específica.

Ambos son cruciales: la función de pérdida guía el entrenamiento al proporcionar una medida de error, mientras que el optimizador ajusta los parámetros del modelo para reducir ese error.


## 4.Entrenamiento del modelo

Durante el entrenamiento de una red neuronal, se alimentan datos de entrada a la red y se calculan las predicciones. La función de pérdida evalúa el error entre las predicciones y las etiquetas verdaderas, y el optimizador ajusta los pesos del modelo para minimizar este error a través de múltiples iteraciones, mejorando así el rendimiento del modelo.

In [65]:
# Hiperparámetros
learning_rate = 0.001
batch_size = 128
epochs = 20

# Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(batch_size)
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

In [66]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [67]:
history = model.fit(train_dataset, 
                    validation_data=validation_dataset, 
                    epochs=epochs)

Epoch 1/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.1924 - loss: 3.0613 - val_accuracy: 0.6365 - val_loss: 1.2809
Epoch 2/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7319 - loss: 0.9314 - val_accuracy: 0.7492 - val_loss: 0.8521
Epoch 3/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8480 - loss: 0.5458 - val_accuracy: 0.7884 - val_loss: 0.7421
Epoch 4/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8945 - loss: 0.3997 - val_accuracy: 0.8231 - val_loss: 0.6396
Epoch 5/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9164 - loss: 0.3158 - val_accuracy: 0.8288 - val_loss: 0.6249
Epoch 6/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9341 - loss: 0.2495 - val_accuracy: 0.8379 - val_loss: 0.6399
Epoch 7/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9450 - loss: 0.2071 - val_accuracy: 0.8481 - val_loss: 0.6285
Epoch 8/20
272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9538 - loss: 0.1798 - val_accuracy: 0.

## 5.Evaluacion del modelo

In [68]:


y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

precision = precision_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

print(f'Precisión: {precision:.2f}')
print(f'F1-Score: {f1:.2f}')



395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Precisión: 0.89
F1-Score: 0.88


In [69]:

# Hacer predicciones con el modelo
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convertir las predicciones a clases

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Generar el reporte de clasificación
class_report = classification_report(y_test, y_pred_classes)

# Presentar los resultados
print("Matriz de Confusión:")
print(conf_matrix)
print("\nReporte de Clasificación:")
print(class_report)

395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Matriz de Confusión:
[[ 31  29   0 ...   0   0   0]
 [  3 690   4 ...   8   0   0]
 [  0  67 648 ...   0   0   0]
 ...
 [  0   6   0 ...  42   0   0]
 [  0   0   0 ...   0  42   4]
 [  0   0   0 ...   0   0  86]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.51      0.52      0.51        60
           1       0.77      0.96      0.85       720
           2       0.92      0.86      0.89       750
           3       0.80      0.92      0.86       450
           4       0.91      0.85      0.88       660
           5       0.89      0.87      0.88       630
           6       0.95      0.79      0.87       150
           7       0.85      0.90      0.87       450
           8       0.88      0.79      0.83       450
           9       0.97      0.92      0.94       480
          10       0.95      0.97      0.96       660
          11       0.89      0.92      0.90       420
          12    

### Interpretación 

- **Historial de Entrenamiento**:
  - **Pérdida (`loss`)**: Disminuye constantemente, indicando que el modelo está mejorando en los datos de entrenamiento.
  - **Precisión (`accuracy`)**: Aumenta significativamente, mostrando que el modelo está haciendo mejores predicciones.

- **Métricas Finales**:
  - **Precisión Global**: 0.89, lo que significa que el 89% de las predicciones son correctas.
  - **F1-Score**: 0.88, reflejando un buen equilibrio entre precisión y recall.

- **Matriz de Confusión**:
  - Muestra que el modelo es eficaz en la clasificación de la mayoría de las clases, aunque hay algunas clases con mayor confusión.

En general, el modelo está funcionando bien, con alta precisión y un buen equilibrio en las métricas de rendimiento.
